In [2]:
#read tweets text file into python
file = open('tweets.txt', 'r')
data = file.read()

In [3]:
#split the text file into a list of tweets
newdata = data.split('\n')

In [4]:
#at this point the tweets are represented as strings, so we use json.loads to convert them to dictionaries
import simplejson
tweetlist=[]
for tweet in newdata[0:275000]:
    tweetlist.append(simplejson.loads(tweet))

In [6]:
#get rid of all retweets, which always start with "RT @"
tweets = []
for i in tweetlist:
    if i['text'][0:4] != 'RT @':
        tweets.append(i)

In [7]:
#create list of swear words to search for in tweets
#source of swear word list: http://www.slate.com/blogs/lexicon_valley/2013/09/11/top_swear_words_most_popular_curse_words_on_facebook.html
swearlist=['fuck', 'FUCK', 'Fuck', 'shit', 'SHIT', 'Shit', 'bitch', 'BITCH', 'Bitch', 'damn', 'DAMN','Damn','dick','DICK','Dick', 'crap', 'CRAP', 'Crap', 'fag','FAG','Fag','piss','Piss', 'PISS','pussy', 'PUSSY','Pussy', 'asshole','ASSHOLE','Asshole','slut','SLUT','Slut', 'cock','COCK','Cock','darn','DARN','Darn','douche','DOUCHE','Douche','bastard','BASTARD', 'Bastard']

In [11]:
#create list of emojis to search for in tweets
#source of emoji list: https://github.com/seandolinar/socialmediaparse/blob/master/data/emoji_table.txt
emoji_key = pd.read_csv('emojis.txt', encoding='utf-8', index_col=0)
emojilist = list(emoji_key.index)

In [12]:
#create list of emoticons to search for in tweets
#source of emoticon list: most of "Western" section of https://en.wikipedia.org/wiki/List_of_emoticons
emoticonlist = [u':‑)', u':)', u':D', u':o)', u':]', u':3', u':c)', u':> =]', u'8)', u'=)', u':}', u':^)', u':っ)', u':‑D', u'8‑D', u'8D', u'x‑D', u'xD', u'X‑D', u'XD', u'=‑D', u'=D', u'=‑3', u'=3', u'B^D', u':-))',u'>:[', u':‑(', u':(', u':‑c', u':c', u':‑<', u':っC', u':<', u':‑[', u':[', u':{', u';(', u':-||', u':@', u'>:(', u':\'‑(', u':\'(', u':\'‑)', u':\')', u'<3', u'</3']

In [43]:
import pandas as pd

#create function that takes in list of tweets and outputs pandas dataframe
#columns tweetID, text, favorite_count, retweet_count, and followers_count come directly from twitter data
#column hashtags is created by creating a list of hashtags for each tweet
#all count oolumns are created by summing up the number of occurrences of the given features
#swear_count, emoji_count, and emotion_count columns are created by searching for items from above lists in each tweet
#hillary, bernie, and martin columns are the response variables and are created by storing a 1 if the candidate is mentioned in that tweet, and a 0 otherwise
def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    DataSet['tweetID'] = [tweet["id"] for tweet in tweets]
    DataSet["text"] = [tweet["text"] for tweet in tweets]
    DataSet["hashtags"] = [([tw["text"] for tw in tweet["entities"]["hashtags"]] if len(tweet["entities"]["hashtags"]) >=1 else None) for tweet in tweets]
    DataSet["hashtag_count"] = [len(tweet["entities"]["hashtags"]) for tweet in tweets]
    DataSet["url_count"] = [len(tweet["entities"]["urls"]) for tweet in tweets]
    DataSet["favorite_count"] = [tweet["favorite_count"] for tweet in tweets]
    DataSet["retweet_count"] = [tweet["retweet_count"] for tweet in tweets]
    DataSet["followers_count"] = [tweet["user"]["followers_count"] for tweet in tweets]
    DataSet["swear_count"] = [len([i for i in swearlist if i in tweet["text"]]) for tweet in tweets]
    DataSet["emoji_count"] = [sum([tweet["text"].count(i) for i in emojilist]) for tweet in tweets]
    DataSet["emoticon_count"] = [sum([tweet["text"].count(i) for i in emoticonlist]) for tweet in tweets]
    DataSet["hillary"] = [max([1 if i in tweet["text"] else 0 for i in ["Hillary", "hillary", "HILLARY", "Hilary", "hilary", "HILARY", "clinton", "Clinton", "CLINTON"]]) for tweet in tweets]
    DataSet["bernie"] = [max([1 if i in tweet["text"][0] else 0 for i in ["Bernie", "bernie", "BERNIE", "Sanders", "sanders", "SANDERS", "Bern", "bern", "BERN"]]) for tweet in tweets]
    DataSet["martin"] = [max([1 if i in tweet["text"][0] else 0 for i in ["Martin", "martin", "MARTIN", "Omalley", "OMalley", "omalley", "OMALLEY", "O'malley", "O'Malley", "o'malley", "O'MALLEY"]]) for tweet in tweets]
    
    return DataSet

#run function on our list of tweets
tweetsdf = toDataFrame(tweets)

#save dataframe to csv file for ease of use in the future
tweetsdf.to_csv('TweetsDF.csv', encoding = 'utf-8')

In [44]:
tweetsdf

,tweetID,text,hashtags,hashtag_count,url_count,favorite_count,retweet_count,followers_count,swear_count,emoji_count,emoticon_count,hillary,bernie,martin
0,665775625974603776,"As campaign focus turns to foreign policy, Cli...",None,0,1,1,0,5949,0,0,0,1,0,0
1,665775612733227008,Hillary Clinton's 10 Most Hawkish Moments From...,None,0,1,0,0,78,0,0,0,1,0,0
2,665775608123826177,Watch The Full Second Democratic Debate Here -...,None,0,1,0,0,156,0,0,0,0,0,0
3,665775602365018113,"Clinton plays defense on Iraq, Wall Street at ...",[empresa],1,1,0,0,725,0,0,0,1,0,0
4,665775598996852736,Democratic Candidates Respond to Paris Attacks...,"[demdebate, Nigeria, Ghana, USA]",4,1,0,0,478,0,0,0,1,0,0
5,665775596983685120,#CNN Hillary Clinton at Democratic debate: ISI...,[CNN],1,1,0,0,1587,0,0,0,1,0,0
6,665775595528192000,Democratic Candidates Respond to Paris Attacks...,"[demdebate, Nigeria, Ghana, USA]",4,1,0,0,1894,0,0,0,1,0,0
7,665775590880907265,Both Hillary and Ronda got beat up tonight. Ba...,"[DemDebate, UFC193]",2,0,6,2,486,0,0,0,1,0,0
8,665775588590776320,Democratic Candidates Respond to Paris Attacks...,"[demdebate, Nigeria, Ghana, USA]",4,1,0,0,364,0,0,0,1,0,0
9,665775585046585345,Democratic Candidates Respond to Paris Attacks...,"[demdebate, breakingnews, India]",3,1,0,0,1894,0,0,0,1,0,0


In [1]:
#read in csv with dataframe that already has sentiment scores added in
newdf = pd.read_csv("finalsentimentedit.csv")

#code to find defective rows (cases where tweet has been split over multiple rows and tweetID is not equal to the tweetID)
for k, i in zip(newdf["tweetID"], newdf.index.values) :
    if len(str(k)) != 8:
        print k, i
#manually went into excel and fixed these rows (~10-15 rows)

/Users/rneuman/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [2]:
#the csv file for newdf was created before response variables were added in, so they are added in here now
newdf["hillary"]= [max([1 if i in j else 0 for i in ["Hillary", "hillary", "HILLARY", "Hilary", "hilary", "HILARY", "clinton", "Clinton", "CLINTON"]]) for j in newdf["text"]]
newdf["bernie"] = [max([1 if i in j else 0 for i in ["Bernie", "bernie", "BERNIE", "Sanders", "sanders", "SANDERS", "Bern", "bern", "BERN"]]) for j in newdf["text"]]
newdf["martin"] = [max([1 if i in j else 0 for i in ["Martin", "martin", "MARTIN", "Omalley", "OMalley", "omalley", "OMALLEY", "O'malley", "O'Malley", "o'malley", "O'MALLEY"]]) for j in newdf["text"]]

In [3]:
# get list of column names to delete first column
list(newdf.columns.values)

# delete additional index column
del newdf['Unnamed: 0']

In [4]:
#look at percentage of tweets that talk about more than one candidate; 22076/106497 ~20%
counter=0
total=0
for h, b, m in zip(newdf["hillary"], newdf["bernie"], newdf["martin"]):
    total+=1
    if h+b+m > 1:
        #print h+b+m
        counter +=1
print counter
print total

22076
106497


In [5]:
#remove tweets that refer to multiple candidates and save as new dataframe
finaldf=newdf[newdf["hillary"] + newdf["bernie"] + newdf["martin"] <= 1]

In [6]:
len(finaldf["bernie"])

84421

In [7]:
#check to make sure all tweets that referred to multiple candidates were removed
total=0
counter=0
for h, b, m in zip(finaldf["hillary"], finaldf["bernie"], finaldf["martin"]):
    total+=1
    if h+b+m > 1:
        counter +=1
print counter
print total

0
84421


In [8]:
#import final modified dataframe from csv
finaldf.to_csv('finaldf.csv')

In [9]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

In [10]:
finaldf.head()

,tweetID,text,hashtags,hashtag_count,url_count,favorite_count,retweet_count,followers_count,swear_count,emoji_count,emoticon_count,sentiment,sentiment_var,hillary,bernie,martin
0,6.66E+17,"As campaign focus turns to foreign policy, Cli...",NaN,0,1,1,0,5949,0,0,0,6.135000,0.015075,1,0,0
1,6.66E+17,Hillary Clinton's 10 Most Hawkish Moments From...,NaN,0,1,0,0,78,0,0,0,6.213333,0.008089,1,0,0
2,6.66E+17,Watch The Full Second Democratic Debate Here -...,NaN,0,1,0,0,156,0,0,0,6.500000,0.320000,0,0,0
3,6.66E+17,"Clinton plays defense on Iraq, Wall Street at ...",[empresa],1,1,0,0,725,0,0,0,5.646667,1.552089,1,0,0
4,6.66E+17,Democratic Candidates Respond to Paris Attacks...,"[demdebate, Nigeria, Ghana, USA]",4,1,0,0,478,0,0,0,5.475000,3.084675,1,0,0


In [11]:
#separate data into three dataframes, one for each candidate
berniedf = finaldf[finaldf["bernie"] == 1]
hillarydf = finaldf[finaldf["hillary"] == 1]
martindf = finaldf[finaldf["martin"] == 1]

In [12]:
berniedf.head()

,tweetID,text,hashtags,hashtag_count,url_count,favorite_count,retweet_count,followers_count,swear_count,emoji_count,emoticon_count,sentiment,sentiment_var,hillary,bernie,martin
16,6.66E+17,CBS News debate changes after Paris attacks pr...,NaN,0,1,0,0,434,0,0,0,4.66,4.8400,0,1,0
18,6.66E+17,Sanders aide pushes back against CBS switch to...,NaN,0,1,0,0,106,0,0,0,4.76,1.8496,0,1,0
30,6.66E+17,Here is the statistics for the 2nd #DemDebate ...,[DemDebate],1,1,1,0,331,0,0,0,7.20,0.0000,0,1,0
31,6.66E+17,All polls point to Bernie winning the debate.#...,"[DemDebate, FeelTheBern]",2,0,2,2,85,0,0,0,7.13,0.8281,0,1,0
32,6.66E+17,330592 - At the second Democratic Party presid...,NaN,0,1,0,0,8484,0,0,0,6.95,0.3969,0,1,0


In [11]:
# average favorite count 
bernie = np.mean(finaldf["favorite_count"][finaldf["bernie"] ==1])
hillary = np.mean(finaldf["favorite_count"][finaldf["hillary"]==1])

In [21]:
#bar chart

# percentage of tweets that contained an emoji, emoticon, swearword, url, retweet, or favorite
emoji_pct_bernie = len((berniedf[berniedf["emoji_count"] >= 1]).index)/float(len(berniedf.index))
emoji_pct_hillary = len((hillarydf[hillarydf["emoji_count"] >= 1]).index)/float(len(hillarydf.index))

emoticon_pct_bernie = len((berniedf[berniedf["emoticon_count"] >= 1]).index)/float(len(berniedf.index))
emoticon_pct_hillary = len((hillarydf[hillarydf["emoticon_count"] >= 1]).index)/float(len(hillarydf.index))

swear_pct_bernie = len((berniedf[berniedf["swear_count"] >= 1]).index)/float(len(berniedf.index))
swear_pct_hillary = len((hillarydf[hillarydf["swear_count"] >= 1]).index)/float(len(hillarydf.index))

url_pct_bernie = len((berniedf[berniedf["url_count"] >= 1]).index)/float(len(berniedf.index))
url_pct_hillary = len((hillarydf[hillarydf["url_count"] >= 1]).index)/float(len(hillarydf.index))

retweet_pct_bernie = len((berniedf[berniedf["retweet_count"] >= 1]).index)/float(len(berniedf.index))
retweet_pct_hillary = len((hillarydf[hillarydf["retweet_count"] >= 1]).index)/float(len(hillarydf.index))

fav_pct_bernie = len((berniedf[berniedf["favorite_count"] >= 1]).index)/float(len(berniedf.index))
fav_pct_hillary = len((hillarydf[hillarydf["favorite_count"] >= 1]).index)/float(len(hillarydf.index))

fig, ax = plt.subplots()

width = .35

groupB = (emoji_pct_bernie, emoticon_pct_bernie, swear_pct_bernie, url_pct_bernie, retweet_pct_bernie, fav_pct_bernie)
groupH = (emoji_pct_hillary, emoticon_pct_hillary, swear_pct_hillary, url_pct_hillary, retweet_pct_hillary, fav_pct_hillary)

index = np.arange(len(groupB))

groupBplot = plt.bar(index, groupB, width, color = "lightsteelblue", label = "Bernie")
groupHplot = plt.bar(index + width, groupH, width, color = 'yellowgreen', label = "Hillary" )

plt.xlabel("Feature")
plt.ylabel("Percentage of Tweets about Candidate That Contain Feature")
plt.title("Comparing Candidate Tweets by Tweet Characteristics")

plt.xticks(index + width, ('Emoji', 'Emoticon', 'Swear', 'Url', 'Retweet', 'Favorite'))
plt.legend(loc ="upper left")

plt.show()

In [15]:
print fav_pct_bernie 
print fav_pct_hillary

0.379533607682
0.323535949811


In [83]:
#look at which swear words crop up the most
swearlist=['fuck', 'FUCK', 'Fuck', 'shit', 'SHIT', 'Shit', 'bitch', 'BITCH', 'Bitch', 'damn', 'DAMN','Damn','crap', 'CRAP', 'Crap', 'fag','FAG','Fag','piss','Piss', 'PISS','pussy', 'PUSSY','Pussy', 'asshole','ASSHOLE','Asshole','slut','SLUT','Slut', 'cock','COCK','Cock','darn','DARN','Darn','douche','DOUCHE','Douche','bastard','BASTARD', 'Bastard']
sweardict_bernie={}
sweardict_hillary={}
for word in swearlist:
    sweardict_bernie[word] = 0
    sweardict_hillary[word] = 0

for text in berniedf.text:
    for word in swearlist:
        if word in text:
            sweardict_bernie[word] += 1
for text in hillarydf.text:
    for word in swearlist:
        if word in text:
            sweardict_hillary[word] += 1

finalsweardictH={}
finalsweardictB={}
finalswearlist=['fuck', 'shit', 'bitch', 'damn']
finalsweardictH['fuck'] = sweardict_hillary['fuck'] + sweardict_hillary['Fuck'] + sweardict_hillary['FUCK']
finalsweardictB['fuck'] = sweardict_bernie['fuck'] + sweardict_bernie['Fuck'] + sweardict_bernie['FUCK']
finalsweardictH['shit'] = sweardict_hillary['shit'] + sweardict_hillary['Shit'] + sweardict_hillary['SHIT']
finalsweardictB['shit'] = sweardict_bernie['fuck'] + sweardict_bernie['Shit'] + sweardict_bernie['SHIT']
finalsweardictH['bitch'] = sweardict_hillary['bitch'] + sweardict_hillary['Bitch'] + sweardict_hillary['BITCH']
finalsweardictB['bitch'] = sweardict_bernie['bitch'] + sweardict_bernie['Bitch'] + sweardict_bernie['BITCH']
finalsweardictH['damn'] = sweardict_hillary['damn'] + sweardict_hillary['Damn'] + sweardict_hillary['DAMN']
finalsweardictB['damn'] = sweardict_bernie['damn'] + sweardict_bernie['Damn'] + sweardict_bernie['DAMN']

for word in finalsweardictH:
    finalsweardictH[word] = finalsweardictH[word]/float(sum(sweardict_hillary.values()))

for word in finalsweardictB:
    finalsweardictB[word] = finalsweardictB[word]/float(sum(sweardict_bernie.values()))

In [86]:
#create graph for swear words in Bernie and Hillary tweets

swearB = finalsweardictB.values()
swearH = finalsweardictH.values()

fig, ax = plt.subplots()

index = np.arange(len(swearB))

width = .35

groupBplot = plt.bar(index, swearB, width, color = "lightsteelblue", label = "Bernie")
groupHplot = plt.bar(index + width, swearH, width, color = 'yellowgreen', label = "Hillary" )


plt.xlabel("Swear Word")
plt.ylabel("Percentage of Swear Word Tweets about Candidate That Contain Swear Word")
plt.title("Comparing Candidates by Swear Words")

plt.xticks(index + width, ('fuck', 'shit', 'bitch', 'damn'))
plt.legend(loc ="upper left")

plt.show()
